In [1]:
import pandas as pd
import numpy as np

In [2]:
base = r"J:\Shared drives\TMD_TSA\Model\networks\Transit\gtfs\mbta_2019\4_imported_gtfs"
gtfs_base = r"J:\Shared drives\TMD_TSA\Model\networks\Transit\gtfs\mbta_2019\3_gtfs_post_move_stops"

base = r"J:\Shared drives\TMD_TSA\Model\networks\Transit\gtfs\bnrd\4_imported_gtfs"
gtfs_base = r"J:\Shared drives\TMD_TSA\Model\networks\Transit\gtfs\bnrd\2_gtfs_py"

In [3]:
tcad_routes = pd.read_csv(base + r"\Transit Routes.csv", low_memory = False)

trips = pd.read_csv(gtfs_base + r"\trips.txt", low_memory = False)
stop_times = pd.read_csv(gtfs_base + r"\stop_times.txt", low_memory = False)

In [4]:
def get_start_stop_times(stop_times):    
    '''for every trip, grab the start time and stop time of the trip
    
    Parameters
    -----------
    stop_times : df
        gtfs stop_times.txt in df format

    Returns
    --------
        flintstone : df
            df with start and stop times per trip

    '''
    chocula =0 
    for trip_id in stop_times['trip_id'].unique():
        max_row = stop_times.query('trip_id==@trip_id').query('stop_sequence == stop_sequence.max()')[['trip_id','arrival_time']]
        min_row = stop_times.query('trip_id==@trip_id').query('stop_sequence == stop_sequence.min()')[['trip_id','arrival_time']]
        r2 = min_row.merge(max_row, how='left', on='trip_id', suffixes = ('_start','_end'))
        if chocula == 0:
            flintstone = pd.DataFrame(r2)
        else:
            flintstone=pd.concat([flintstone,r2])
        chocula +=1
    return(flintstone)


In [5]:
def assign_tod(start_stop):
    '''calculate midpoint of trip, use midpoint to assign TOD
    
    Parameters
    -----------
    start_stop : df
        df with start and stop times per trip

    Returns
    --------
    start_stop :
        df with start time, stop time, midpoint time, and TOD per trip

    '''
    
    start_stop['at_end_dec'] = (
        (
            (start_stop['arrival_time_end'].str.split(":").str[0]).astype('int32')
            +
            ((start_stop['arrival_time_end'].str.split(":").str[1]
            ).astype('int32')/60)))
    start_stop['at_start_dec'] = (
        (
            (start_stop['arrival_time_start'].str.split(":").str[0]).astype('int32')
            +
            ((start_stop['arrival_time_start'].str.split(":").str[1]
            ).astype('int32')/60)))
    
    start_stop['midpoint'] = start_stop['at_start_dec'] + ((start_stop['at_end_dec']-start_stop['at_start_dec'])/2)
    start_stop['tod'] = np.where(start_stop['midpoint'].between(6.50,9.50),'AM', np.where(
        start_stop['midpoint'].between(9.50,15.00), 'MD', np.where(
            start_stop['midpoint'].between(15.00,19.00),'PM', 'NT' 
        )
            ) 
        )
    
    return start_stop


In [6]:
start_stop = get_start_stop_times(stop_times) # simpson

In [7]:
start_stop_tod = assign_tod(start_stop) # smurf

In [8]:
fred = ["Red","Orange","Blue","Green-E","Green-B","Green-C","Green-D","Mattapan","Boat-F4","CR-Worcester","CR-Kingston"
,"CR-Haverhill","CR-Providence","CR-Newburyport","CR-Needham","CR-Middleborough","CR-Lowell","CR-Greenbush","CR-Franklin"
,"CR-Fitchburg","CR-Fairmount","Boat-F1","Boat-F2","Boat-F3"]

In [9]:
route_name = {}
for t in tcad_routes['Trip']:
    rn = tcad_routes.query('Trip == @t')['Route_Name'].to_list()[0]
    gtfs_n = trips.query('trip_id == @t')['route_pattern_id']
    route_name[rn] = gtfs_n.to_list()[0]
    #group_trips = trips.query('route_pattern_id == @gtfs_n')['trip_id']
    #sst = start_stop_tod.query('trip_id in @group_trips')
    #count_tod = sst.groupby(by='tod').count()


In [10]:
# check if multiple route patterns per route name
reltab = pd.DataFrame.from_dict(route_name, orient='index', columns = ['rpid']).reset_index()
reltab.groupby('rpid').count().query('index > 1')

,index
rpid,


In [11]:
trips2 = trips.merge(reltab, how='left', left_on = 'route_pattern_id',right_on = 'rpid').rename(columns={'index':'Route_Name'})

ValueError: You are trying to merge on object and float64 columns. If you wish to proceed you should use pd.concat

In [ ]:
len(trips['route_pattern_id'].unique())

In [ ]:
len(trips.query('route_id not in @fred')['route_pattern_id'].unique())

In [ ]:
len(tcad_routes['Route_Name'].unique())

In [ ]:
trips.query('route_pattern_id not in @reltab.rpid').query('route_id not in @fred')['route_pattern_id']

In [ ]:
len(trips.query('route_id =="34E"')['route_pattern_id'].unique())

In [ ]:
len(tcad_routes.query('Route == "34E"'))

In [ ]:
trips34e = trips.query('route_pattern_id == "34E-4-0"')['trip_id']

In [ ]:
start_stop_tod.query('trip_id in @trips34e').groupby('tod').count()

In [ ]:
tod_trips = trips2.merge(start_stop_tod).groupby(by=['Route_Name','tod']).agg({'trip_id':'count'}).rename(columns = {'trip_id':'num_trips'}).reset_index()

In [ ]:
tod_pivot = tod_trips.pivot(index='Route_Name', columns = 'tod', values = 'num_trips').reset_index()
tod_pivot

In [ ]:
print("AM: "+str(tod_pivot['AM'].sum()))
print("MD: "+str(tod_pivot['MD'].sum()))
print("PM: "+str(tod_pivot['PM'].sum()))
print("NT: "+str(tod_pivot['NT'].sum()))

In [ ]:
trips2.merge(start_stop_tod).query('route_id not in @fred').groupby('tod').count()

In [ ]:
tod_pivot.to_csv(base+r"\tod_pivot.csv")